<a href="https://colab.research.google.com/github/hairymax/Python-for-science-lecture-notes/blob/main/03_SciPy/3.3.exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scipy : Самостоятельные упражения

В упражнениях используются в основном Numpy, Scipy и Matplotlib. Они предоставляют несколько реальных примеров научных вычислений с помощью Python.

Данные для заданий могут быть скачаны из [репозитория GitHub](https://github.com/hairymax/Python-for-science-lecture-notes/tree/main/data) 

## Задание 1. Прогноз максимальной скорости ветра на станции Спрогё

Оригинальный текст: [Maximum wind speed prediction at the Sprogø station](https://scipy-lectures.org/intro/summary-exercises/stats-interpolate.html)

*Цель упражнения* - предсказать максимальную зафиксированную скорость ветра, наблюдающуюся на протяжении 50 лет, даже если за такой период времени не существует никаких измерений. Имеющиеся данные измерены только за 21 год на метеорологической станции Спрогё (Sprogø), расположенной в Дании. Сначала будут приведены статистические шаги, а затем проиллюстрированы с помощью функций из модуля `scipy.interpolate`. Предлагается получить результаты на основе необработанных данных и с использованием несколько иного подхода.

### Статистический подход

Предполагается, что годовые максимумы должны соответствовать нормальной функции плотности вероятности. Однако такая функция не сможет быть подобрана, поскольку она дает вероятность максимума скорости ветра. Нахождение максимальной скорости ветра, возникающей каждые 50 лет, требует противоположного подхода, результат нужно найти по известной вероятности. Для этого используют квантильную функцию, целью упражнения будет ее нахождение. В текущей модели предполагается, что максимальная скорость ветра, возникающая каждые 50 лет, определяется как верхний 2% квантиль.

По определению, квантильная функция является обратной функцией кумулятивного распределения. Последняя описывает распределение вероятности нахождения годового максимума. В упражнении кумулятивная вероятность $p_i$ для данного года $i$ определяется как $p_i = i/(N+1)$, где $N = 21$ - количество измеренных лет. Таким образом, можно рассчитать кумулятивную вероятность каждого измеренного максимума скорости ветра. На этих экспериментальных точках с помощью модуля `scipy.interpolate` можно подогнать квантильную функцию. Наконец, максимальное значение за 50 лет будет оценено по кумулятивной вероятности 2% квантиля.

### Вычисление кумулятивных вероятностей
Годовые максимумы скорости ветра уже были вычислены и сохранены в формате `numpy` в файле `examples/max-speeds.npy`, поэтому они будут загружены с помощью numpy:

>> 
``` python
import numpy as np
max_speeds = np.load('examples/max-speeds.npy')
years_nb = max_speeds.shape[0]
```
Следуя определению кумулятивной вероятности `p_i` из предыдущего раздела, соответствующие значения будут:
>> 
``` python
cprob = (np.arange(years_nb, dtype=np.float32) + 1)/(years_nb + 1)
```
и предполагается, что они соответствуют заданным скоростям ветра:
>>
``` python
sorted_max_speeds = np.sort(max_speeds)
```

### Прогнозирование с помощью `UnivariateSpline`
В этом разделе квантильная функция будет cnhjbnmcz с помощью класса `UnivariateSpline`, который может строить сплайн по точкам. По умолчанию строится сплайн 3 степени, сами точки могут иметь различные веса в зависимости от их достоверности. Вариации этого метода `InterpolatedUnivariateSpline` и `LSQUnivariateSpline`, в которых изменена проверка ошибок. Для построения двумерных сплайнов используется семейство классов `BivariateSpline`. Все эти классы для одномерных и двумерных сплайнов используют подпрограммы *FITPACK Fortran*, поэтому для построения сплайна возможен более низкоуровневый доступ к библиотеке через функции `splrep` и `splev`. Для упрощения использования функции интерполяции реализованы без использования параметров FITPACK (см. `interp1d`, `interp2d`, `barycentric_interpolate` и т.д.).

Для максимальных скоростей ветра в Спрогё будет использоваться `UnivariateSpline`, поскольку сплайн 3 степени, позволяет достаточно точно описать данные:

>>
``` python
from scipy.interpolate import UnivariateSpline
quantile_func = UnivariateSpline(cprob, sorted_max_speeds)
```
Теперь квантильная функция будет оцениваться по полному диапазону вероятностей:

>>
``` python
nprob = np.linspace(0, 1, 1e2)
fitted_max_speeds = quantile_func(nprob)
```
В текущей модели максимальная скорость ветра, возникающая каждые 50 лет, определяется как верхний 2% квантиль. В результате значение кумулятивной вероятности составит:

>>
``` python
fifty_prob = 1. - 0.02
```
Таким образом, скорость штормового ветра, возникающего каждые 50 лет, можно определить следующим образом:

>>
``` python
fifty_wind = quantile_func(fifty_prob)
fifty_wind      
```
Результаты можно отобразить с помощью Matplotlib:

![](https://scipy-lectures.org/_images/sphx_glr_plot_cumulative_wind_speed_prediction_001.png)

### Упражнение с распределением Гумбелла
Эту задачу можно решить другим способом: используя данные о скорости ветра, измеренные за 21 год. Период измерений составляет около 90 минут (изначально - 10 минут, но для уменьшения размера файла и упрощения упражнения данные сгруппированы по 90 минут). Данные хранятся в формате numpy в файле `'data/sprog-windspeeds.npy'`.

- Первым шагом необходимо найти годовые максимумы с помощью `numpy` и построить их в виде гистограммы matplotlib.
  
![](https://scipy-lectures.org/_images/sphx_glr_plot_sprog_annual_maxima_001.png)

- На втором шаге необходимо использовать распределения Гумбелла для кумулятивных вероятностей `p_i`, определяемого как `-log( -log(p_i) )` для подгонки линейной квантильной функции (вы можете определить степень `UnivariateSpline`). Далее строится график годовых максимумов в зависимости от кумулятивной вероятности Гумбелла, который должен иметь вид.

![](https://scipy-lectures.org/_images/sphx_glr_plot_gumbell_wind_speed_prediction_001.png)

- Последним шагом определяем значение 34,23 м/с для максимальной скорости ветра, возникающей каждые 50 лет.

## Задание 2. Приближение кривой методом наименьших квадратов: извлечение точек из топографических лидарных данных

Оригинальный текст: [Non linear least squares curve fitting: application to point extraction in topographical lidar data](https://scipy-lectures.org/intro/summary-exercises/optimize-fit.html)

*Цель этого упражнения* - подобрать модель к некоторым данным. Данные, используемые в этом уроке, измерены лидаром.

Системы лидаров - это оптические дальномеры, которые измеряют расстояние на основе анализа рассеянного света. Большинство из них испускают короткий световой импульс в направлении цели и регистрируют отраженный сигнал. Затем этот сигнал обрабатывается для определения расстояния между лидаром и целью.

Топографические лидарные системы - это такие системы, встраиваемые в воздушные платформы. Они измеряют расстояния между платформой и Землей, чтобы получить информацию о рельефе Земли (подробнее см. в <sup><sub>Mallet, C. and Bretar, F. Full-Waveform Topographic Lidar: State-of-the-Art. ISPRS [Journal of Photogrammetry and Remote Sensing 64(1), pp.1-16, January 2009](http://dx.doi.org/10.1016/j.isprsjprs.2008.09.007)</sub></sup>). Данные, используемые здесь, являются частью демонстрационных данных, доступных для программного обеспечения [FullAnalyze](http://fullanalyze.sourceforge.net/)

В данном задании целью является анализ формы волны, регистрируемой лидарной системой. Такой сигнал содержит пики, центр и амплитуда которых позволяют вычислить положение и некоторые характеристики облучаемой цели. Когда след лазерного луча составляет около 1 м на поверхности Земли, луч может поразить несколько целей (например, землю и верхушку дерева или здания). Сумма вкладов каждой цели создает сложный сигнал с несколькими пиками, каждый из которых содержит информацию об одной цели.

Одним из современных методов извлечения информации из этих данных является разложение их в сумму гауссовых функций, где каждая функция представляет собой вклад цели, облучённой лазерным лучом.

Будем использовать модуль `scipy.optimize` для представления формы волны в виде одной или суммы гауссовых функций.

### Загрузка и визуализация

Загрузите первую осциллограмму, используя:
>>
``` python
import numpy as np
waveform_1 = np.load('../data/waveform_1.npy')
```
и визуализируйте ее:
>>
``` python
import matplotlib.pyplot as plt
t = np.arange(len(waveform_1))
plt.plot(t, waveform_1) 
plt.show()
```
Форма волны представляет собой сигнал размером 80 точек с одним пиком с амплитудой около 30 при 15 наносекундах. Базовый уровень шума составляет примерно 3. Эти значения можно использовать в исходном решении.

![](https://scipy-lectures.org/_images/sphx_glr_plot_optimize_lidar_data_001.png)

### Нахождение формы сигнала с помощью простой гауссовой модели

Сигнал очень прост и может быть смоделирован как одна гауссова функция со смещением, соответствующем фоновому шуму. Чтобы подогнать сигнал под функцию, мы должны:
- определить модель
- определить начальное решение
- вызвать `scipy.optimize.leastsq`

**Модель**

Гауссовская функция, заданная выражением
$$B + A \exp\left\{-\left(\frac{t-\mu}{\sigma}\right)^2\right\}$$
может быть определена на python следующим образом:

>>
``` python
def model(t, coeffs):
   return coeffs[0] + coeffs[1] * np.exp( - ((t-coeffs[2])/coeffs[3])**2 )
```
где
- `coeffs[0]` - $B$ (шум)
- `coeffs[1]` - $A$ (амплитуда)
- `coeffs[2]` - $\mu$ (центр)
- `coeffs[3]` - $\sigma$ (ширина).
 
**Начальное решение**

Одно из возможных начальных решений, следующее:

>>
``` python
x0 = np.array([3, 30, 15, 1], dtype=float)
```
**Подгонка**

`scipy.optimize.leastsq` минимизирует сумму квадратов функции, заданной в качестве аргумента. Минимизируемая функция - это разница между данными и моделью:

>>
``` python
def residuals(coeffs, y, t):
    return y - model(t, coeffs)
```
Получим решение, вызвав `scipy.optimize.leastsq()` со следующими аргументами:
- функция для минимизации
- начальное решение
- дополнительные аргументы для передачи в функцию
>>
``` python
from scipy.optimize import leastsq
t = np.arange(len(waveform_1))
x, flag = leastsq(residuals, x0, args=(waveform_1, t))
print(x)
```
И визуализируем решение:
``` python
fig, ax = plt.subplots(figsize=(8, 6))
plt.plot(t, waveform_1, t, model(t, x))
plt.xlabel('Время [нс]')
plt.ylabel('Амплитуда [бины]')
plt.legend(['Waveform', 'Model'])
plt.show()
```
![](https://scipy-lectures.org/_images/sphx_glr_plot_optimize_lidar_data_fit_001.png)

Замечание: начиная со scipy v0.8 и выше, лучше использовать `scipy.optimize.curve_fit()`, которая принимает модель и данные в качестве аргументов, поэтому не нужно определять разности между данными и моделью.

### Дополнительно

Попробуйте использовать более сложную форму волны (файл `'data/waveform_2.npy'`), которая содержит три значимых пика. Вы должны адаптировать модель, которая теперь представляет собой сумму гауссовых функций вместо одной.

![](https://scipy-lectures.org/_images/sphx_glr_plot_optimize_lidar_complex_data_001.png)

- В некоторых случаях написать явную функцию для вычисления якобиана быстрее, чем искать его численно с помощью `leastsq`. Создайте функцию для вычисления якобиана остатков и используйте ее в качестве входных данных для `leastsq`.`
- Когда мы хотим обнаружить очень маленькие пики в сигнале, или когда начальное предположение слишком далеко от хорошего решения, результат, выдаваемый алгоритмом, часто не получается неудовлетворительным. Добавление ограничений на параметры модели позволяет преодолеть эти недостатки. Примером априорных знаний, которые мы можем добавить, является знак наших переменных (которые все положительные).
- *Дополнительное упражнение*: сравните результат работы `scipy.optimize.leastsq()` и результат `scipy.optimize.fmin_slsqp()` с добавлением граничных условий.

## Задача 3. Обработка изображений: подсчет пузырьков и нерасплавленных зерен

Оригинальный текст: [Image processing application: counting bubbles and unmolten grains](https://scipy-lectures.org/intro/summary-exercises/image-processing.html)

![](https://scipy-lectures.org/_images/MV_HFV_012.jpg)

- Откройте файл изображения `MV_HFV_012.jpg` и отобразите его. Найдите в справке аргументы метода `imshow`, которые необходимо указать, чтобы отобразить изображение с "правильной" ориентацией (начало координат в левом нижнем углу, а не в левом верхнем, как для стандартных массивов).  
На этом изображении, полученном с растрового электронного микроскопа, показан образец стекла (светло-серая матрица) с некоторыми пузырьками (на черном фоне) и нерасплавленными песчинками (темно-серый цвет). Необходимо определить долю образца, покрытую этими тремя фазами, а также оценить типичный размер песчинок и пузырьков, их размеры и т.д.

- Обрежьте изображение, чтобы удалить нижнюю панель с информацией об измерении.

- Отфильтруйте изображение с помощью медианного фильтра, чтобы уточнить его гистограмму. Проверьте, как изменилась гистограмма.

- Используя гистограмму отфильтрованного изображения, определите пороговые значения, позволяющие определить маски для пикселей песка, пикселей стекла и пикселей пузырьков.  
  Другой вариант: написать функцию, которая автоматически определяет пороговые значения по минимумам гистограммы.

- Выведите на экран изображение, на котором три фазы окрашены тремя разными цветами.

- Используйте математическую морфологию для очистки различных фаз.

- Припишите метки всем пузырькам и песчинкам и удалите из маски песчинки, размер которых меньше 10 пикселей. Для этого используйте `ndimage.sum` или `np.bincount` для вычисления размеров зерен.

- Вычислите средний размер пузырьков.